# Generating Audio with LSTM 

This notebook will let you download a song from Youtube and model it with an **LSTM**

This models the way each spectral frame follows another and can be used to generate new raw audio

## Install prerequisites and get code

In [ ]:
%tensorflow_version 1.x
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # less warnings ...
import tensorflow as tf

In [ ]:
!git clone https://github.com/ual-cci/music_gen_interaction_RTML.git

In [ ]:
# python libraries
!pip install Pillow numpy opencv-python PyWavelets tqdm slugify
!pip install -U Flask
!pip install lws==1.2.6
!pip install tflearn
!pip install librosa==0.7.2
!pip install numba==0.48
!pip install mock

In [ ]:
%cd /content/music_gen_interaction_RTML

## Download a sample audio:

Note: replace the url with whatever music video you want - or upload a file directly ... You can use the ffmpeg to convert it to wav later.

In [ ]:
# get a youtube downloader
!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

# Set up training

In [ ]:
%cd /content/music_gen_interaction_RTML/

from unittest.mock import Mock, MagicMock
args = MagicMock(name='method')
sample_rate = 22050

# This will set the same setting for the training and the running of the model:
args.lstm_layers = 3
args.lstm_units = 128
args.sample_rate = sample_rate
args.sequence_length = 40
args.async_loading = True
args.amount_epochs = -1

# Pick a new song from Youtube 

## Insert your own url below and train 



In [ ]:
youtube_url = "https://www.youtube.com/watch?v=4cIWu5m8UmA"

!youtube-dl -ci -f "bestaudio[ext=m4a]" $youtube_url -o 'youtube_audio.m4a'
!ffmpeg -i 'youtube_audio.m4a' -ac 2 -f wav full.wav

In [ ]:
# Audio file preparation - this cuts a 1 minute sample from the audio file:
!ffmpeg -y -ss 60 -i full.wav -t 60 -c copy sample.wav

### You will probably need to train for 300 epochs

In [9]:
number_of_epochs = 150 # will take cca 4min
number_of_epochs = 300 # will take cca 8min

song_name = "sample.wav" # < we will train the model on this wav file
model_name = "my_trained_model" # < and then save the model under this name (check the .tfl files in the folder afterwards)

# Train Model

In [ ]:
# ----[keep the same bellow]-------------------------------------------------------------------

# takes time!
!python training_handler.py -target_file $song_name -model_name $model_name -amount_epochs $number_of_epochs -batch_size 512 \
                            -lstm_layers $args.lstm_layers  -lstm_units $args.lstm_units -sample_rate $args.sample_rate -sequence_length $args.sequence_length
from IPython.display import clear_output 
clear_output()

import glob
import numpy as np
i = glob.glob("*.tfl.png")
from IPython.display import Audio, Image
display(Image(i[0]))

# Generate

As Vit mentioned in the lecture, we start from a place in the original audio track, then use the model to keep predicting new audio frames. 

Unfortunately, sometimes it can get stuck in a loop, or just stop generating interesting things so we can give it a kick by jumping to a new point in the song. 

Once we've switched, we keep using the model to generate new audio frames!

Below, you can specify a sequence of lengths and changes to the generation process to create a new audio output. 

The `sequence` array stores these points. Each item in is an array that stores the **start position** and **segment length**. The default has three, but you can add in as many as you want and/or change the existing values. 

```
sequence = [
  [start1, length1],
  [start2, length2],
  [start3, length3],
  etc....
]
```


The default code:

* Starts generating 10% through the song and generates 200 frames 

* Moves to 60% through the song and generates 300 frames

* Moves to 90% through the song and generates 150 frames

**Try with your own!**

In [11]:
# Here you can change the paths to other trained models / other song files
load_generative_seeds_from = song_name
load_model_from = model_name+".tfl"

In [12]:
from server_handler import ServerHandler
import settings

my_settings = settings.Settings(args)
print("Important Settings: settings.lstm_layers=", my_settings.lstm_layers, ", settings.lstm_units=", my_settings.lstm_units,
              ", settings.sample_rate=", my_settings.sample_rate)

generation_handler = ServerHandler(my_settings, manual_loading = True)
generation_handler.manual_init_song_model(load_generative_seeds_from, load_model_from)

# slightly experimental interpolation through the latents while generating ...

generation_handler.change_impulse(0.2) # set to 20% sharp

sequence = [
  #Starts generating 10% through the song and generates 200 frames 
  [0.1, 200],
  #Moves to 60% through the song and generates 300 frames
  [0.6, 300],
  #Moves to 90% through the song and generates 150 frames
  [0.9, 150]            
]

output_audio = []

for i in sequence:
  position_in_the_song = i[0]
  requested_length = i[1]
  generation_handler.change_impulse_smoothly_start(position_in_the_song) # allow interpolation
  audio, predict, reconstruct = generation_handler.generate_audio_sample(requested_length, interactive_i=position_in_the_song)
  output_audio.append(audio)

clear_output()

# Play Audio

If you want to download the audio, you can use the Colab File Explorer on the left <----. 

Find the file `generated_output_exp_concat.wav` and select **download**

In [ ]:
import librosa
from IPython.display import Audio, Image
output_audio = np.concatenate(output_audio)

out_name = 'generated_output_exp_concat.wav'
librosa.output.write_wav(out_name, output_audio, sr=sample_rate)
Audio(out_name)

# Clean up if you are going to train a new model from a different youtube video

In [14]:
# fast cleanup
!mkdir unused
!mv *.wav unused/
!mv *.m4a unused/